In [75]:
from bs4 import BeautifulSoup
import datetime
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
import re

In [44]:
today = datetime.datetime.today().strftime('%d.%m.%Y')

In [76]:
file_path: str = f'/Users/dmitrijdolgopolov/Documents/1_study_python/projects/1_avito_parsing/flats_from_avito/page_codes/Page_code_19.04.2023.txt'

In [93]:
def get_flat_info(url, headless=True):
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-blink-features=AutomationControlled')
    if headless:
        options.add_argument('--headless')
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    try:
        driver.get(url=url)
        time.sleep(5)
        flat_page_code = driver.page_source
        flat_soup = BeautifulSoup(flat_page_code, 'html.parser')
        params_li = flat_soup.find_all('li', class_='params-paramsList__item-appQw')
        flat_square_value, flat_floor, flat_floors_in_house = 1, '-', '-'
        
        params_needed = 3
        count = 0
        for i in range(len(params_li)):
            if any(x in params_li[i].find('span', class_='desktop-3a1zuq')
                               .get_text() for x in ('Общая площадь', 'Площадь комнаты')):
                flat_square_value = float(params_li[i].get_text()
                                                      .replace('Общая площадь', '')
                                                      .replace('Площадь комнаты', '')
                                                      .replace(': ', '')
                                                      .replace('\xa0', ' ')
                                                      .split()[0])
                count += 1
                if count >= 3:
                    break
            elif 'Этаж' in params_li[i].find('span', class_='desktop-3a1zuq').get_text():
                flat_floor = int(params_li[i].get_text()
                                             .replace('Этаж', '')
                                             .replace(': ', '')
                                             .split(' ')[0])
                flat_floors_in_house = int(params_li[i].get_text()
                                                       .replace('Этаж', '')
                                                       .replace(': ', '')
                                                       .split(' ')[-1])
                count += (1 + 1)
                if count >= params_needed:
                    print('успешно завершено.')
                    break
        if count < 3:
            print(f'На странице не хватает параметров: {url}')
    except Exception as ex:
        print(ex)
    finally:
        driver.close()
        driver.quit()
        return flat_square_value, flat_floor, flat_floors_in_house

In [94]:
with open(file_path) as file:
    page_code = file.read()
soup = BeautifulSoup(page_code, 'html.parser')
item_divs = soup.find_all('div', class_='iva-item-body-KLUuy')
for item in item_divs:
        if len(item.find('h3').get_text().split(', ')) >= 2:
            item_floor = item.find('h3') \
                .get_text() \
                .split(', ')[-1] \
                .replace('\xa0', ' ') \
                .split()[0] \
                .split('/')[0]
            item_house_floors = item.find('h3') \
                .get_text() \
                .split(', ')[-1] \
                .replace('\xa0', ' ') \
                .split()[0] \
                .split('/')[-1]
            item_square = item.find('h3') \
                .get_text().split(', ')[1] \
                .replace('\xa0', ' ') \
                .split()[0] \
                .replace(',', '.')
            if (item_floor.isdigit()) and (('.' in item_square) or item_square.isdigit()):
                item_square = float(item_square)
                item_floor, item_house_floors = int(item_floor), int(item_house_floors)
            else:
                flat_url = 'https://www.avito.ru' + item.find('a', class_='link-link-MbQDP').get('href')
                print(f'Неверный тип данных, нужно зайти на страницу квартиры: {flat_url}')
                item_square, item_floor, item_house_floors = get_flat_info(flat_url)
                if any(type(x) == 'str' for x in (item_square, item_floor, item_house_floors)):
                    print(f'Строковое значение {x} в url: {flat_url}')
        else:
            flat_url = 'https://www.avito.ru' + item.find('a', class_='link-link-MbQDP').get('href')
            print(f'Недостаточно информации, нужно зайти на страницу квартиры: {flat_url}')
            item_square, item_floor, item_house_floors = get_flat_info(flat_url)

Недостаточно информации, нужно зайти на страницу квартиры: https://www.avito.ru/sankt-peterburg/kvartiry/kvartira_2613111204


/var/folders/_w/82802rvj0rv88rym_s2j3p6m0000gn/T/ipykernel_65702/231139357.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


успешно завершено.
Неверный тип данных, нужно зайти на страницу квартиры: https://www.avito.ru/sankt-peterburg/komnaty/komnata_194m_v_2-k._512et._2782997124
успешно завершено.


In [104]:
def convert_month(month_name):
    dict_of_months = {'января': '01', 'февраля': '02', 'марта': '03', 'апреля': '04', 'мая': '05', 'июня': '06',
                      'июля': '07', 'августа': '08', 'сентября': '09', 'октября': '10', 'ноября': '11', 'декабря': '12'}
    for el in dict_of_months.keys():
        if month_name in el:
            return dict_of_months[el]

In [112]:
item_date_day = '15 марта 14:33'
date_pattern = r'(\d{1,2})\s(\w+)\s\d{1,2}:\d{1,2}'
re_result = re.findall(date_pattern, item_date_day)
if re_result is not None:
    item_day = re_result[0][0]
    item_month = convert_month(re_result[0][1])
    year = datetime.date.today().year
print(f'{item_day}.{item_month}.{year}')

15.03.2023


In [100]:
?datetime.date.year

Object `datetime.date.year()` not found.


In [67]:
for i in range(len(item_divs)):
    if item_divs[i].find('div', class_='geo-address-fhHd0 text-text-LurtD text-size-s-BxGpL') is None:
        print(item_divs[i])
#     item_address = item_divs[i].find('div', class_='geo-address-fhHd0 text-text-LurtD text-size-s-BxGpL')\
#                            .find('span').get_text()
#     print(item_address)

In [42]:
n = 1
for i in range(len(item_divs)):
    if len(item_divs[i].find('h3').get_text().split(', ')) >= 2:
        item_square = float(item_divs[i].find('h3').get_text().split(', ')[1].split('\xa0')[0].replace(',', '.'))
    else:
        flat_url = 'https://www.avito.ru' + item_divs[i].find('a', class_='link-link-MbQDP').get('href')
        flat_square = get_flat_info(flat_url)
        print(i, flat_square)
        break
#     n += 1
#     if n > 1000:
#         break

/var/folders/_w/82802rvj0rv88rym_s2j3p6m0000gn/T/ipykernel_65702/1897308353.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


243 34.3
